--- Day 16: The Floor Will Be Lava ---

With the beam of light completely focused somewhere, the reindeer leads you deeper still into the Lava Production Facility. At some point, you realize that the steel facility walls have been replaced with cave, and the doorways are just cave, and the floor is cave, and you're pretty sure this is actually just a giant cave.

Finally, as you approach what must be the heart of the mountain, you see a bright light in a cavern up ahead. There, you discover that the beam of light you so carefully focused is emerging from the cavern wall closest to the facility and pouring all of its energy into a contraption on the opposite side.

Upon closer inspection, the contraption appears to be a flat, two-dimensional square grid containing empty space (.), mirrors (/ and \), and splitters (| and -).

The contraption is aligned so that most of the beam bounces around the grid, but each tile on the grid converts some of the beam's light into heat to melt the rock in the cavern.

You note the layout of the contraption (your puzzle input). For example:

.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....

The beam enters in the top-left corner from the left and heading to the right. Then, its behavior depends on what it encounters as it moves:

    If the beam encounters empty space (.), it continues in the same direction.
    If the beam encounters a mirror (/ or \), the beam is reflected 90 degrees depending on the angle of the mirror. For instance, a rightward-moving beam that encounters a / mirror would continue upward in the mirror's column, while a rightward-moving beam that encounters a \ mirror would continue downward from the mirror's column.
    If the beam encounters the pointy end of a splitter (| or -), the beam passes through the splitter as if the splitter were empty space. For instance, a rightward-moving beam that encounters a - splitter would continue in the same direction.
    If the beam encounters the flat side of a splitter (| or -), the beam is split into two beams going in each of the two directions the splitter's pointy ends are pointing. For instance, a rightward-moving beam that encounters a | splitter would split into two beams: one that continues upward from the splitter's column and one that continues downward from the splitter's column.

Beams do not interact with other beams; a tile can have many beams passing through it at the same time. A tile is energized if that tile has at least one beam pass through it, reflect in it, or split in it.

In the above example, here is how the beam of light bounces around the contraption:

>|<<<\....
|v-.\^....
.v...|->>>
.v...v^.|.
.v...v^...
.v...v^..\
.v../2\\..
<->-/vv|..
.|<<<2-|.\
.v//.|.v..

Beams are only shown on empty tiles; arrows indicate the direction of the beams. If a tile contains beams moving in multiple directions, the number of distinct directions is shown instead. Here is the same diagram but instead only showing whether a tile is energized (#) or not (.):

######....
.#...#....
.#...#####
.#...##...
.#...##...
.#...##...
.#..####..
########..
.#######..
.#...#.#..

Ultimately, in this example, 46 tiles become energized.

The light isn't energizing enough tiles to produce lava; to debug the contraption, you need to start by analyzing the current situation. With the beam starting in the top-left heading right, how many tiles end up being energized?

In [ ]:
file_input = open(file="./Inputs/Day16.txt", mode='r')
my_input = file_input.read()

In [ ]:
import enum

class Direction(enum.Enum):
    UP = (-1, 0)
    RIGHT = (0, 1)
    DOWN = (1, 0)
    LEFT = (0, -1)

def isInside(pos, matris):
    return 0 <= pos[0] < len(matris) and 0 <= pos[1] < len(matris[0])

def printTrace(beam_pos, direction, mirrors, where_to_print):
    while isInside(beam_pos, where_to_print) and direction not in where_to_print[beam_pos[0]][beam_pos[1]]:
        mirror_char = mirrors[beam_pos[0]][beam_pos[1]]
        where_to_print[beam_pos[0]][beam_pos[1]].add(direction)    
        if mirror_char == '/':
            if direction == Direction.UP:
                direction = Direction.RIGHT
            elif direction == Direction.RIGHT:
                direction = Direction.UP
            elif direction == Direction.DOWN:
                direction = Direction.LEFT
            elif direction == Direction.LEFT:
                direction = Direction.DOWN    
        
        elif mirror_char == '\\':
            if direction == Direction.UP:
                direction = Direction.LEFT
            elif direction == Direction.LEFT:
                direction = Direction.UP
            elif direction == Direction.DOWN:
                direction = Direction.RIGHT
            elif direction == Direction.RIGHT:
                direction = Direction.DOWN
            
        elif mirror_char == '-':
            if direction == Direction.UP or direction == Direction.DOWN:
                printTrace((beam_pos[0], beam_pos[1]-1), Direction.LEFT, mirrors, where_to_print)
                printTrace((beam_pos[0], beam_pos[1]+1), Direction.RIGHT, mirrors, where_to_print)
                break
            
        elif mirror_char == '|':
            if direction == Direction.LEFT or direction == Direction.RIGHT:
                printTrace((beam_pos[0]-1, beam_pos[1]), Direction.UP, mirrors, where_to_print)
                printTrace((beam_pos[0]+1, beam_pos[1]), Direction.DOWN, mirrors, where_to_print)
                break
        
        beam_pos = (beam_pos[0] + direction.value[0], beam_pos[1] + direction.value[1])

def countEnegized(beam_pos, direction, mirrors):
    where_to_print = [[set() for _ in range(len(mirrors[0]))] for _ in range(len(mirrors))]
    printTrace(beam_pos, direction, mirrors, where_to_print)
    count = 0
    for row in where_to_print:
        for direction_set in row:
            if direction_set:
                count += 1
    return count


mirrors = []
for line in my_input.splitlines():
    mirrors.append(line)

print(countEnegized((0, 0), Direction.RIGHT, mirrors))

--- Part Two ---

As you try to work out what might be wrong, the reindeer tugs on your shirt and leads you to a nearby control panel. There, a collection of buttons lets you align the contraption so that the beam enters from any edge tile and heading away from that edge. (You can choose either of two directions for the beam if it starts on a corner; for instance, if the beam starts in the bottom-right corner, it can start heading either left or upward.)

So, the beam could start on any tile in the top row (heading downward), any tile in the bottom row (heading upward), any tile in the leftmost column (heading right), or any tile in the rightmost column (heading left). To produce lava, you need to find the configuration that energizes as many tiles as possible.

In the above example, this can be achieved by starting the beam in the fourth tile from the left in the top row:

.|<2<\....
|v-v\^....
.v.v.|->>>
.v.v.v^.|.
.v.v.v^...
.v.v.v^..\
.v.v/2\\..
<-2-/vv|..
.|<<<2-|.\
.v//.|.v..

Using this configuration, 51 tiles are energized:

.#####....
.#.#.#....
.#.#.#####
.#.#.##...
.#.#.##...
.#.#.##...
.#.#####..
########..
.#######..
.#...#.#..

Find the initial beam configuration that energizes the largest number of tiles; how many tiles are energized in that configuration?

In [ ]:
max_energized = 0
for i in range(2*len(mirrors[0])+2*len(mirrors)):
    row = -1
    col = -1
    direction = Direction.RIGHT
    if i < len(mirrors[0]):
        row = 0
        col = i
        direction = Direction.DOWN
    elif i < len(mirrors[0])+len(mirrors):
        row = i-len(mirrors[0])
        col = len(mirrors[0])-1
        direction = Direction.LEFT
    elif i < 2*len(mirrors[0])+len(mirrors):
        row = len(mirrors)-1
        col = i-(len(mirrors[0])+len(mirrors))
        direction = Direction.UP
    else:
        row = i-(2*len(mirrors[0])+len(mirrors))
        col = 0
        direction = Direction.RIGHT
    
    max_energized = max(max_energized, countEnegized((row, col), direction, mirrors))
print(max_energized)